# Data analyst agent: get your data's insights in the blink of an eye ✨
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

> This tutorial is advanced. You should have notions from [this other cookbook](agents) first!

In this notebook we will make a **data analyst agent: a Code agent armed with data analysis libraries, that can load and transform dataframes to extract insights from your data, and even plots the results!**

Let's say I want to analyze the data from the [Kaggle Titanic challenge](https://www.kaggle.com/competitions/titanic) in order to predict the survival of individual passengers. But before digging into this myself, I want an autonomous agent to prepare the analysis for me by extracting trends and plotting some figures to find insights.

Let's set up this system.

Run the line below to install required dependancies:

In [ ]:
!pip install seaborn smolagents transformers -q -U

We first create the agent. We used a `CodeAgent` (read the [documentation](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution) to learn more about types of agents), so we do not even need to give it any tools: it can directly run its code.

We simply make sure to let it use data science-related libraries by passing these in `additional_authorized_imports`: `["numpy", "pandas", "matplotlib.pyplot", "seaborn"]`.

In general when passing libraries in `additional_authorized_imports`, make sure they are installed on your local environment, since the python interpreter can only use libraries installed on your environment.

⚙ Our agent will be powered by [meta-llama/Llama-3.1-70B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct) using `HfApiModel` class that uses HF's Inference API: the Inference API allows to quickly and easily run any open model, for free!

In [42]:
from smolagents import HfApiModel, CodeAgent
from huggingface_hub import login
import os

login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

model = HfApiModel("meta-llama/Llama-3.1-70B-Instruct")

agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=["numpy", "pandas", "matplotlib.pyplot", "seaborn"],
    #max_iterations=10,
)

## Data analysis 📊🤔

Upon running the agent, we provide it with additional notes directly taken from the competition, and give these as a kwarg to the `run` method:

In [43]:
import os

os.mkdir("./figures")

FileExistsError: [Errno 17] File exists: './figures'

In [46]:
additional_notes = """
### Variable Notes
SepalWidthCm: Width of Sepal
SepalLengthCm = Length of Sepal
PetalLengthCm = Length of Petal
PetalWidthCm = Width of Petal
Species: Species of the flower , also the value to be predicted
"""

analysis = agent.run(
    """You are an expert data analyst.
Please load the source file and analyze its content.
According to the variables you have, begin by listing 3 interesting questions that could be asked on this data, for instance about specific correlations with survival rate.
Then answer these questions one by one, by finding the relevant numbers.
Meanwhile, plot some figures using matplotlib/seaborn and save them to the (already existing) folder './figures/': take care to clear each figure with plt.clf() before doing another plot.

In your final answer: summarize these correlations and trends
After each number derive real worlds insights, for instance: "Correlation between is_december and boredness is 1.3453, which suggest people are more bored in winter".
Your final answer should have at least 3 numbered and detailed parts.
""",
    additional_args=dict(
        additional_notes=additional_notes,
        source_file="/content/iris_train.csv"
    )
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert data analyst.                                                                                 │
│ Please load the source file and analyze its content.                                                            │
│ According to the variables you have, begin by listing 3 interesting questions that could be asked on this data, │
│ for instance about specific correlations with survival rate.                                                    │
│ Then answer these questions one by one, by finding the relevant numbers.                                        │
│ Meanwhile, plot some figures using matplotlib/seaborn and save them to the (already existing) folder            │
│ './figures/': take care to clear each figure with plt.clf() before doing another plot.                          │
│                                                                                                                 │
│ In your final answer: summarize these correlations and trends                                                   │
│ After each number derive real worlds insights, for instance: "Correlation between is_december and boredness is  │
│ 1.3453, which suggest people are more bored in winter".                                                         │
│ Your final answer should have at least 3 numbered and detailed parts.                                           │
│                                                                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'additional_notes': '\n### Variable Notes\nSepalWidthCm: Width of Sepal\nSepalLengthCm = Length of             │
│ Sepal\nPetalLengthCm = Length of Petal\nPetalWidthCm = Width of Petal\nSpecies: Species of the flower , also    │
│ the value to be predicted\n', 'source_file': '/content/iris_train.csv'}.                                        │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.1-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  df = pd.read_csv(source_file)                                                                                    
  print(df.head())                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0  23            4.6           3.6            1.0           0.2   
1  16            5.7           4.4            1.5           0.4   
2  66            6.7           3.1            4.4           1.4   
3  12            4.8           3.4            1.6           0.2   
4  43            4.4           3.2            1.3           0.2   

           Species  
0      Iris-setosa  
1      Iris-setosa  
2  Iris-versicolor  
3      Iris-setosa  
4      Iris-setosa  

Out: None

[Step 0: Duration 5.68 seconds| Input tokens: 2,268 | Output tokens: 54]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
The code blob is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in code_blob=" The 
correlation between sepal width and petal length is 0.78.\nThe correlation between sepal width and petal length is 
0.78.\nThe correlation between sepal width and petalLengthCм']<end_code>\n\nObservation:". Make sure to include 
code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 5: Duration 3.44 seconds| Input tokens: 20,423 | Output tokens: 2,369]

Reached max steps.

KeyboardInterrupt: 

In [ ]:
print(analysis)

The analysis of the Titanic data reveals that socio-economic status and sex are significant factors in determining survival rates. Passengers with lower socio-economic status and males are less likely to survive. The age of a passenger has a minimal impact on their survival rate.


Impressive, isn't it? You could also provide your agent with a visualizer tool to let it reflect upon its own graphs!

## Data scientist agent: Run predictions 🛠️

👉 Now let's dig further: **we will let our model perform predictions on the data.**

To do so, we also let it use `sklearn` in the `additional_authorized_imports`.

In [ ]:
agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=[
        "numpy",
        "pandas",
        "matplotlib.pyplot",
        "seaborn",
        "sklearn",
    ],
    max_iterations=12,
)

output = agent.run(
    """You are an expert machine learning engineer.
Please train a ML model on "titanic/train.csv" to predict the survival for rows of "titanic/test.csv".
Output the results under './output.csv'.
Take care to import functions and modules before using them!
""",
    additional_args=dict(additional_notes=additional_notes + "\n" + analysis)
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert machine learning engineer.                                                                    │
│ Please train a ML model on "titanic/train.csv" to predict the survival for rows of "titanic/test.csv".          │
│ Output the results under './output.csv'.                                                                        │
│ Take care to import functions and modules before using them!                                                    │
│                                                                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'additional_notes': '\n### Variable Notes\npclass: A proxy for socio-economic status (SES)\n1st = Upper\n2nd = │
│ Middle\n3rd = Lower\nage: Age is fractional if less than 1. If the age is estimated, is it in the form of       │
│ xx.5\nsibsp: The dataset defines family relations in this way...\nSibling = brother, sister, stepbrother,       │
│ stepsister\nSpouse = husband, wife (mistresses and fiancés were ignored)\nparch: The dataset defines family     │
│ relations in this way...\nParent = mother, father\nChild = daughter, son, stepdaughter, stepson\nSome children  │
│ travelled only with a nanny, therefore parch=0 for them.\n\nThe analysis of the Titanic data reveals that       │
│ socio-economic status and sex are significant factors in determining survival rates. Passengers with lower      │
│ socio-economic status and males are less likely to survive. The age of a passenger has a minimal impact on      │
│ their survival rate.'}.                                                                                         │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.1-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 import pandas as pd                                                                                         │
│   2 import numpy as np                                                                                          │
│   3 from sklearn.model_selection import train_test_split                                                        │
│   4 from sklearn.ensemble import RandomForestClassifier                                                         │
│   5 from sklearn.metrics import accuracy_score                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out: None

[Step 0: Duration 23.22 seconds| Input tokens: 2,238 | Output tokens: 185]

...

[Step 5: Duration 16.43 seconds| Input tokens: 20,624 | Output tokens: 1,614]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer("The model has been trained and the predictions have been saved to a CSV file named            │
│     'output.csv'.")                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: The model has been trained and the predictions have been saved to a CSV file named 
'output.csv'.

[Step 6: Duration 3.00 seconds| Input tokens: 26,342 | Output tokens: 1,677]

Even though the agent got a few errors, it managed to correctly solve the problem in the end!

The test predictions that the agent output above, once submitted to Kaggle, score **0.78229**, which is #2824 out of 17,360, and better than what I had painfully achieved when first trying the challenge years ago.

Your result will vary, but anyway I find it very impressive to achieve this with an agent in a few seconds.

🚀 The above is just a naive attempt with agent data analyst: it can certainly be improved a lot to fit your use case better!